<a href="https://colab.research.google.com/github/MohanZhu0623/Sentiment_Analysis/blob/main/Combined_TM(positive).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install contextualized-topic-models

In [2]:
%%capture
!pip install pyldavis

In [3]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

In [4]:
!head -n 2 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and


In [5]:
import pandas as pd
df = pd.read_excel('/content/kickstarter_data_sampled_sentiment&WC.xlsx')
text_data = df[df['sentiment'] == 1]['blurb'].tolist()

In [6]:
# Print the number of rows
print(f'Number of rows: {len(text_data)}')

Number of rows: 9026


In [7]:
!pip install --upgrade scipy gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 56.1 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 4.2.0
    Uninstalling gensim-4.2.0:
      Successfully uninstalled gensim-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contextualized-topic-models 2.5.0 requires gensim==4.2.0, but you have gensim 4.3.3 which is incompatible.


In [8]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:
from nltk.corpus import stopwords as stop_words

nltk.download('stopwords')

documents = [line.strip() for line in text_data]

stopwords = list(stop_words.words("english"))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation

tp = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [11]:
tp.vocab[:10]

array(['ability', 'able', 'access', 'accessible', 'accessories',
       'accessory', 'accurate', 'achieve', 'acoustic', 'across'],
      dtype=object)

In [12]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=10)
ctm.fit(training_dataset)   # run the model

0it [00:00, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch: [10/10]	 Seen Samples: [89600/90170]	Train Loss: 68.00289966038295	Time: 0:00:04.033235: : 10it [00:33,  3.34s/it]
100%|██████████| 141/141 [00:02<00:00, 59.25it/s]


In [13]:
# Get the top 10 words for each topic
topic_lists = ctm.get_topic_lists(10)

# Output the top 10 words for each topic line by line
for i, topic in enumerate(topic_lists):
    print(f'Topic {i + 1}: {", ".join(topic)}')

Topic 1: create, us, help, people, bring, community, want, art, fund, project
Topic 2: adventure, light, story, journey, festival, celebrate, earth, celebration, comedy, another
Topic 3: community, food, bring, want, family, local, truck, dream, help, farm
Topic 4: help, us, get, need, make, like, want, fund, would, first
Topic 5: album, songs, record, new, first, recording, ready, rock, part, band
Topic 6: people, world, share, community, art, learn, social, young, platform, life
Topic 7: life, music, live, dance, world, real, positive, art, young, together
Topic 8: made, designed, leather, handmade, quality, handcrafted, unique, hand, shoes, custom
Topic 9: album, new, songs, music, part, help, record, debut, ready, first
Topic 10: film, interactive, adventure, fun, world, positive, spirit, magazine, creative, beauty
Topic 11: book, children, beautiful, art, fun, kids, limited, illustrated, images, photography
Topic 12: free, candles, business, online, app, money, get, make, every, s

In [14]:
topic_lists = ctm.get_topic_lists(5)
print(topic_lists)

[['create', 'us', 'help', 'people', 'bring'], ['adventure', 'light', 'story', 'journey', 'festival'], ['community', 'food', 'bring', 'want', 'family'], ['help', 'us', 'get', 'need', 'make'], ['album', 'songs', 'record', 'new', 'first'], ['people', 'world', 'share', 'community', 'art'], ['life', 'music', 'live', 'dance', 'world'], ['made', 'designed', 'leather', 'handmade', 'quality'], ['album', 'new', 'songs', 'music', 'part'], ['film', 'interactive', 'adventure', 'fun', 'world'], ['book', 'children', 'beautiful', 'art', 'fun'], ['free', 'candles', 'business', 'online', 'app'], ['game', 'cards', 'playing', 'card', 'deck'], ['music', 'new', 'songs', 'world', 'original'], ['natural', 'delicious', 'ingredients', 'candles', 'hot'], ['food', 'truck', 'delicious', 'coffee', 'fresh'], ['help', 'need', 'make', 'us', 'album'], ['enamel', 'pins', 'inspired', 'hard', 'pin'], ['features', 'technology', 'portable', 'power', 'phone'], ['smart', 'easy', 'wireless', 'phone', 'anywhere']]


In [16]:
lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

100%|██████████| 141/141 [00:03<00:00, 36.03it/s]


In [17]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

  0%|          | 0/141 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 141/141 [00:02<00:00, 63.00it/s]
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
